In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import time
import os

import sampler

In [2]:
resnet= models.resnet50(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 75)
resnet.load_state_dict(torch.load('./model75.pth'))

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/ma-user/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [14:15<00:00, 119881.15it/s]


In [3]:
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 25)

In [4]:
dataTrans = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
            transforms.RandomCrop(96),
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
 
    # image data path
def loaddata():
    data_dir = './images2/images2'
    all_image_datasets = datasets.ImageFolder(data_dir, dataTrans)
    
    trainsize = int(0.9*len(all_image_datasets))
    testsize = len(all_image_datasets) - trainsize
    train_dataset, test_dataset = torch.utils.data.random_split(all_image_datasets,[trainsize,testsize])
    
    image_datasets = {'train':train_dataset,'val':test_dataset}
    

    # wrap your data and label into Tensor
    dataloders = {
        'train':torch.utils.data.DataLoader(train_dataset,
                                        batch_size=64,
                                        shuffle=True,
                                        num_workers=4,
                                       # sampler=sampler.ImbalancedDatasetSampler(train_dataset.dataset)
                                                      ),
        'val':torch.utils.data.DataLoader(test_dataset,
                                        batch_size=64,
                                              shuffle=True,
                                              num_workers=4)
            }
    
    dataloders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                 batch_size=64,
                                                 shuffle=True,
                                                 num_workers=4) for x in ['train', 'val']}

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    return dataloders,dataset_sizes
    # use gpu or not
use_gpu = torch.cuda.is_available()

In [5]:
def train_model(model, lossfunc, optimizer, scheduler, num_epochs=10):
    start_time = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
       # dataloders,dataset_sizes=loaddata()
        print(dataloders['val'])
        print(len(dataloders['val']))

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0

            # Iterate over data.
            for data in dataloders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = lossfunc(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data).to(torch.float32)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    elapsed_time = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(
        elapsed_time // 60, elapsed_time % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
  
    return model


In [6]:
dataloders,dataset_sizes=loaddata()

In [ ]:
if use_gpu:
    resnet = resnet.cuda()

    # define loss function
lossfunc = nn.CrossEntropyLoss()

    # setting optimizer and trainable parameters
params = list(resnet.parameters())
#params = list(resnet.fc.parameters())+list(resnet.layer4.parameters())
optimizer_ft = optim.SGD(params, lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model(model=resnet,
                           lossfunc=lossfunc,
                           optimizer=optimizer_ft,
                           scheduler=exp_lr_scheduler,
                           num_epochs=40)

Epoch 0/39
----------
13
train Loss: 0.0403 Acc: 0.3482
val Loss: 0.0384 Acc: 0.3862
Epoch 1/39
----------
13
train Loss: 0.0339 Acc: 0.4211
val Loss: 0.0331 Acc: 0.4491
Epoch 2/39
----------
13
train Loss: 0.0293 Acc: 0.4938
val Loss: 0.0283 Acc: 0.5409
Epoch 3/39
----------
13
train Loss: 0.0257 Acc: 0.5499
val Loss: 0.0243 Acc: 0.6038
Epoch 4/39
----------
13
train Loss: 0.0230 Acc: 0.5950
val Loss: 0.0223 Acc: 0.6176
Epoch 5/39
----------
13


In [ ]:
torch.save(resnet.state_dict(), './model25.pth') 